In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers, callbacks

img_size = 128
batch_size = 32
epochs = 30

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_data = train_datagen.flow_from_directory(
        r'C:\Users\abc\Desktop\Final Project\Plant_Disease_Prediction\sugarcanedata\train',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
      r'C:\Users\abc\Desktop\Final Project\Plant_Disease_Prediction\sugarcanedata\train',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # Freeze base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    callbacks=[
        callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        callbacks.ModelCheckpoint('trained_plant_disease_model2.keras', save_best_only=True)
    ]
)


Found 2177 images belonging to 6 classes.
Found 542 images belonging to 6 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


C:\Users\abc\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5209 - loss: 1.3089

C:\Users\abc\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


69/69 ━━━━━━━━━━━━━━━━━━━━ 212s 3s/step - accuracy: 0.5225 - loss: 1.3042 - val_accuracy: 0.6125 - val_loss: 0.9436
Epoch 2/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.7445 - loss: 0.6514 - val_accuracy: 0.6771 - val_loss: 0.8917
Epoch 3/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.8009 - loss: 0.5151 - val_accuracy: 0.6827 - val_loss: 0.7820
Epoch 4/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 158s 2s/step - accuracy: 0.7999 - loss: 0.5239 - val_accuracy: 0.7011 - val_loss: 0.7609
Epoch 5/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.8219 - loss: 0.4739 - val_accuracy: 0.6956 - val_loss: 0.7302
Epoch 6/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.8311 - loss: 0.4466 - val_accuracy: 0.7362 - val_loss: 0.7296
Epoch 7/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.8378 - loss: 0.4361 - val_accuracy: 0.7288 - val_loss: 0.7425
Epoch 8/30
69/69 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.8364 - loss: 0.4740 - val_accuracy: 0.6956 - val_loss: 0.77

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load pretrained base model without top layers
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(128, 128, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model initially

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Setup Data Generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2  # reserve 20% for validation
)

train_generator = train_datagen.flow_from_directory(
     r"C:\Users\abc\Desktop\Final Project\Plant_Disease_Prediction\sugarcanedata\train",
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
     r"C:\Users\abc\Desktop\Final Project\Plant_Disease_Prediction\sugarcanedata\valid",
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Train initial head
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Unfreeze some base layers and fine-tune
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_finetune = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Save model
model.save('improved_plant_disease_model.keras')
